# <center> GLM-4-Video-9b 介绍与部署流程

>前言

GLM-4-Voice 是智谱 AI 推出的端到端语音模型。GLM-4-Voice 能够直接理解和生成中英文语音，进行实时语音对话，并且能够遵循用户的指令要求改变语音的情感、语调、语速、方言等属性。

>目录

一 GLM-4-Video-9b 模型介绍<br>
二 多模态与AGI技术展望<br>
三 本地部署流程<br>
四 线上部署流程<br>
五 开源大模型能力比较<br>

# 1. GLM-4-Video-9b 模型介绍

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101125504929.png" width=80%></div>

信息来源，智谱官网：https://zhipuai.cn/news/141?tab=1


2024年10月19日，智谱A推出的GLM-4-Voice是一款**端到端**情感语音模型，能流畅地理解并生成中英文语音，为用户带来实时对话体验。GLM-4-Voice不仅能够精准响应用户指令，还可灵活调整语音的情感、语调、语速和方言等多种特征，打造更加自然、丰富的交流效果。

GLM-4-Voice由三大核心模块组成：首先，GLM-4-Voice-Tokenizer将连续语音转换为离散token，实现更高效的语音信息处理；接着，GLM-4-Voice-Decoder将token还原为连续语音输出，保持音质的自然顺滑；最后，GLM-4Voice-9B模型基于GLM-4-9B进行了预训练和对齐，赋予系统更强的语音理解和生成能力。

通过端到端建模，GLM-4-Voice减少了信息损失，显著提升了交互的自然度和流畅性。支持低延迟实时响应的GLM-4-Voice，带来丰富、多维度的语音交互体验，开启智能语音新时代！

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101104227547.png" width=80%></div>

智谱A推出的GLM-4-Voice，让智能语音交流进入全新境界：
- 理解和生成语音:直接理解和生成中英文语音，实现流畅的人机交互

- 丰富的情感表达：从欢欣到忧伤，从愤怒到恐惧，GLM-4-Voice让声音传递更真实的情感共鸣，捕捉情绪的细腻变化。

- 可控语速：在对话中自由调整语速，快些慢些随心所欲，为用户提供更贴合情境的语速调节。

- 灵活指令响应：对话中支持随时打断，根据用户指令调整输出内容和风格，真正实现灵活互动。

- 多语言和多方言支持：全面支持中英文及中国多地方言，包括粤语、重庆话、北京话等，让不同地域的用户都能感受到亲切的交流体验。



**GLM-4-Voice模型核心架构**

与传统的语音识别、语言模型和语音合成（ASR + LLM + TTS）级联方案相比，GLM-4-Voice使用端到端建模方法，以离散音频token形式直接表征语音信号，在单一模型中实现了语音的理解与生成。

>传统的 ASR（Automatic Speech Recognition）、LLM（Large Language Model）、TTS（Text-to-Speech）级联方案是一种分阶段的系统结构，用于语音交互任务。该方案分为三个独立模块：
>1. ASR（语音识别）：首先将用户的语音转换为文本。该模块通过语音识别模型提取语音信号中的语义信息，生成对应的文本表示，但在此阶段中，原始语音的副语言信息（如语速、情感等）通常无法保留。
>2. LLM（大型语言模型）：处理由 ASR 提供的文本并生成适当的回复。LLM 在接收到文字后进行自然语言理解、推理和生成，但该过程只处理纯文本信息，不具备原始语音的语气、情感等信息。
>3. TTS（文本转语音合成）：将 LLM 生成的文本回复转化为语音。TTS 模型根据文本内容合成语音回复，通常使用固定的语调和情感风格，因此无法自适应 ASR 传递的原始语音中的个性化音色和语气。
在这种方案中，三个模块相互依赖，必须依次完成，带来以下主要问题：
>- 延迟：三个模块顺序执行，尤其是在生成语音时需要等待 ASR 和 LLM 完成，导致对话系统的响应时间增加。
>- 信息丢失：ASR 将语音转为文本后，语音中重要的情感、语气和其他副语言信息可能丢失，TTS 难以恢复这些信息，从而使生成的语音听起来较为机械化。
>- 误差传播：ASR、LLM 和 TTS 模块间的误差可能逐步累积，特别是 ASR 和 TTS 的解码质量影响整体对话质量。

- 一个以传统的 ASR（自动语音识别） + LLM（大型语言模型） + TTS（文本到语音合成）级联方案进行构建的例子是 “Whisper — ChatGPT4” 系统。
在这个系统中，首先使用 Whisper 模型将用户的语音输入转换为文本（ASR），然后将生成的文本输入到 ChatGPT4（LLM）进行理解和处理，最后再通过 TTS 模型将生成的文本转换为语音输出。这种级联方法虽然有效，但可能会在语音转文本和文本转语音的过程中引入信息损失，影响整体的交互体验

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101110544330.png" width=80%></div>

GLM-4-Voice的架构基于音频的离散token化表示，实现了音频输入和输出的完整端到端建模。具体而言，通过有监督方式训练的音频Tokenizer基于语音识别模型构建，能够在12.5Hz的低采样频率（对应每秒12.5个音频token）下精确保留语义信息，还包含语速、情感等副语言信息，使语音交流更加自然流畅。在语音合成方面，GLM-4-Voice采用流匹配（Flow Matching）模型，可以在需要最少仅10个token时流式合成语音，从而大幅降低对话延迟，提高响应速度。

为克服模型在语音模态下的理解能力和合成表现力的难点，GLM-4-Voice的预训练任务将Speech2Speech任务拆分为两个子任务：Speech2Text（根据用户音频生成文本回复）和Text2Speech（根据文本内容结合用户语音合成回复语音），并针对这两个任务设计了两种预训练目标。Speech2Text任务利用大规模文本数据，通过将文本句子转化为音频token来训练模型；Text2Speech任务则基于无监督的音频数据和音频-文本转录数据，从音频中随机选择句子，以支持文本转录后的语音合成能力。这一解耦合方案提升了模型对语音理解和生成的适应性和表现力，为端到端的语音处理开创了新的路径。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101111009316.png" width=80%></div>

GLM-4-Voice的训练基于基座模型GLM-4-9B模型，在经过数百万小时的音频数据和数千亿token的音频-文本交错数据的深度预训练，展现出卓越的音频理解和建模能力。为实现高质量的实时语音对话，GLM-4-Voice特别设计了流式思考架构：在接收用户语音输入后，模型能够以流式方式交替生成文本和语音输出。在语音模态中，模型以文本输出作为参照，以确保回复内容的准确性，并能够根据用户的语音指令灵活调整语音表达，从而实现符合用户需求的声音变化。并具有端到端建模 Speech2Speech 的能力，同时保证低延迟性（最低只需要输出 20 个 token 便可以合成语音）。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101111540891.png" width=80%></div>

- 端到端模型是一种在单一模型架构中同时处理任务的输入和输出的深度学习模型。在语音处理任务中，端到端模型能够直接将原始输入（如语音信号）映射到目标输出（如生成的语音或文本）而无需分阶段的多模块处理。
对于语音任务，端到端模型通常具备以下特点：
1. 简化架构：端到端模型整合了 ASR、LLM 和 TTS 的功能，无需使用多个模块逐步处理输入和输出。这种方式减少了多个模块之间的依赖和误差传播问题，避免了传统级联方案中不可避免的延迟和信息丢失。
2. 联合优化：端到端模型可以直接优化整个任务的目标（如准确性或自然性），不需要分步优化各个子模块。这使得模型在语音生成任务上更自然流畅，因为它能够捕获语速、情感等副语言信息，并在生成语音时保留这些信息。
3. 直接映射输入到输出：端到端模型可以从输入端（如语音信号）直接生成所需的输出（如文本或生成的语音），无需先进行语音识别再生成文本或语音。这意味着它能根据整体的输入语音特征（如音色和情感）生成自然的语音回复。

拓展阅读：https://ar5iv.labs.arxiv.org/html/2212.04356?_immersive_translate_auto_translate=1

# 2. 多模态与AGI技术展望

>多模态（Multimodality）

多模态技术是指系统能够同时处理来自多种不同模态（如文本、图像、音频、视频等）的数据，以提供更丰富的信息交互和更精确的理解能力。在多模态模型中，每种模态的信息相互补充，共同帮助模型更全面地理解环境、语境和用户需求。典型的多模态应用包括图像生成和标注、视频分析、语音识别与合成等。这些应用通过多模态信息的融合，大大提升了模型的表现和可用性。

随着模型架构和预训练方法的不断发展，多模态技术已在逐步朝着统一表示的方向发展，以实现模态间无缝转化和信息共享。例如，一些先进模型通过将视觉和文本信息映射到相同的语义空间，使得模型不仅能够理解图片内容，还能够生成相应的描述甚至回答复杂问题。

展望未来，多模态技术将进一步推动人机交互的自然性，特别是在智能助手、虚拟现实（VR）、增强现实（AR）等领域。我们可以期待更多应用将多模态技术与情境感知结合，让机器更全面地理解人类需求，从而提供更个性化、情感化的服务。

>人工通用智能（AGI, Artificial General Intelligence）

AGI指的是一种超越特定任务局限，具备通用智能的人工智能系统。AGI的目标是让机器不仅能完成指定任务，还能像人类一样具备自主学习、推理、适应和创造的能力。与当前的AI系统不同，AGI能跨模态、跨任务进行知识迁移，并在未知情境中表现出灵活性和创造力。例如，一台AGI系统不仅能识别图像和理解语言，还能在复杂环境中自动推理、决策并进行创新。

AGI的实现涉及多个关键挑战，包括模型的可塑性、认知与情感的理解、自主性和安全性等。当前的AI系统大多基于特定任务进行训练，缺乏通用性和灵活性，尤其在处理跨领域任务时会遇到瓶颈。然而，随着大规模模型训练、深度学习和多模态融合等技术的不断进步，AGI的实现变得越来越有可能。

展望未来，AGI的潜力是巨大的：它将为医疗、教育、科研、工程等领域带来变革性的突破，同时也可能带来一系列伦理与安全问题。因此，研究人员需要在AGI发展的过程中严格遵循安全、透明和道德规范，以确保其在造福社会的前提下安全发展。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101113151693.png" width=80%></div>

GLM-4-Voice的推出标志着智谱在通向AGI的道路上实现了又一关键进展。

在工具使用方面，2024年10月28日，智谱推出了创新的AutoGLM功能，旨在显著扩展模型的能力边界。为了提升模型与用户日常生活的贴合度，智谱特别引入了AutoGLM的phone use能力，使其能通过简单的文字或语音指令模拟人类在手机上的操作。理论上，AutoGLM能够完成任何人类可在电子设备上执行的操作，不再局限于简单的任务或API调用，也无需用户搭建复杂的工作流，其操作逻辑与人类相似。

AutoGLM的基础架构建立在智谱自研的“基础智能体解耦合中间界面”及“自进化在线课程强化学习框架”之上。核心技术WebRL解决了大模型智能体在任务规划和动作执行方面的难题，如训练任务和数据稀缺、反馈信号不足、策略分布漂移等挑战，并通过自适应学习策略在迭代中不断优化自身性能，从而在长时间训练中保持性能的持续提升，类似于人类在成长过程中不断积累新技能。

AutoGLM的性能在Phone Use和Web Browser Use上取得了显著提升。特别是在AndroidLab评测基准上，AutoGLM超越了GPT-4o与Claude-3.5-Sonnet的表现；在WebArena-Lite评测基准中，其性能相对GPT-4o提升了约200%，缩小了智能体在GUI操作中的成功率与人类的差距。当前，AutoGLM Web已通过“智谱清言”插件对外开放，支持在网站上自动执行高级检索、内容总结与生成等复杂任务。

从单一文本模态拓展至图像、视频和情感语音模型，AutoGLM的背后依托于智谱最新的基座模型GLM-4-Plus。GLM-4-Plus在语言和文本处理方面的表现已与GPT-4o及405B参数量的Llama3.1相当。近年来，基于GLM-4-Plus，智谱在多模态模型领域取得了显著进展，本次发布的GLM-4-Voice进一步完善了GLM多模态模型家族，使其在迈向原生多模态模型的方向上更进一阶。

>大脑是一个极为复杂的系统，具备多种感知和认知能力，包括听觉、视觉、语言等多模态的感知与理解功能，以及短期和长期记忆的管理。它还拥有深度思考与推理的能力，并能生成丰富的情感和想象力。作为人体的中枢指挥器官，大脑不仅能够对外部刺激做出复杂的响应，还能够灵活调动身体的各个部位，实现精细的协同运作，同时具备使用工具的能力。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101114517666.png" width=80%></div>

面向AGI的发展路径，实际上就是逐步复现人类大脑各项能力的技术升级曲线。从2021年起，尤其在最近一年中，大模型在L1语言能力方面已取得显著突破，达到了人类语言理解能力的80%-90%水平。

随着技术向更高层次发展，单一语言能力的提升已不再足够。当前的目标是实现跨模态的原生多模态模型，使其不仅在认知层面能够比肩人类，还能在价值观层面保持对齐，确保人工智能的安全与可控。是当下这个阶段大模型技术公司需要努力的方向。

作为L3工具能力探索的一部分，AutoGLM可以看作智谱在智能体操作方面的重要尝试，旨在推动人机交互范式的变革，并为构建GLM-OS——即以大模型为核心的通用计算系统——奠定基础。AutoGLM的长远目标是让大模型具备与人类相似的工具能力，能够自主感知环境、规划任务，并执行动作（如使用工具和软件），从而高效完成复杂任务。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101115725365.png" width=50%></div>

AutoGLM仅需简单的文字或语音指令，便可实现对手机操作的模拟，完成一系列实用任务。例如，AutoGLM能够帮助用户在微信上为老板的朋友圈点赞并发表评论、在淘宝上重新购买历史订单中的特定产品，还可以在携程上预订酒店、在12306上购票、或在美团上下单外卖。

凭借对图形用户界面（GUI）的深刻理解，AutoGLM理论上能够实现人类在各类可视化设备（如电脑、手机、平板）上的任何操作，而不受限于简单的任务场景或API调用。此外，它无需用户手动搭建繁琐的工作流，而是通过与人类相似的操作逻辑自然地辅助用户在日常生活和工作中完成各类任务，从而显著提升人机交互的便捷性与灵活性。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101115759302.png" width=60%></div>

针对GUI智能体训练中轨迹数据极度稀缺且成本高昂的问题，以及在复杂任务和真实环境中所需的灵活规划与即时纠正能力，智谱研发了一种“自进化在线课程强化学习框架”，以Web浏览器为实验环境，逐步提升大模型智能体在网络与手机环境中的自主学习与操作能力。该框架通过自进化学习策略和课程强化学习方法，根据智能体的当前能力动态调整任务难度，使模型在自我探索中充分发挥潜力。同时，通过KL散度控制的策略更新与置信度经验回放机制，避免了训练中模型遗忘先前学习任务的现象。

# 3. 本地部署流程

>以下操作都是在命令行实现的。

- **Step 1. 创建conda虚拟环境**

&emsp;&emsp;Conda创建虚拟环境的意义在于提供了一个隔离的、独立的环境，用于Python项目和其依赖包的管理。每个虚拟环境都有自己的Python运行时和一组库。这意味着我们可以在不同的环境中安装不同版本的库而互不影响。建议Python版本3.10以上。创建虚拟环境的办法可以通过使用以下命令创建：

```bash
# glm_4v 是你想要给环境的名称，python=3.11 指定了要安装的Python版本。你可以根据需要选择不同的名称和/或Python版本。

conda create -n glm_4v python=3.11
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028170904630.png" width=80%></div>

&emsp;&emsp;创建虚拟环境后，需要激活它。使用以下命令来激活刚刚创建的环境。如果成功激活，可以看到在命令行的最前方的括号中，就标识了当前的虚拟环境。虚拟环境创建完成后接下来安装torch。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171026651.png" width=80%></div>

&emsp;&emsp;如果忘记或者想要管理自己建立的虚拟环境，可以通过conda env list命令来查看所有已创建的环境名称。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143512354.png" width=80%></div>

&emsp;&emsp;如果需要卸载指定的虚拟环境则通过以下指令实现：
```
conda env remove --name envname
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143643575.png" width=80%></div>

- 需要注意的是无法卸载当前激活的环境，建议卸载时先切换到base环境中再执行操作。

- **Step 2. 查看当前驱动最高支持的CUDA版本**

&emsp;&emsp;我们需要根据CUDA版本选择Pytorch框架，先看下当前的CUDA版本：
```
nvidia -smi
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924161818464.png" width=70%></div>

- **Step 3. 在虚拟环境中安装Pytorch**

&emsp;&emsp;进入Pytorch官网：https://pytorch.org/get-started/previous-versions/

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240103163206436.png" width=80%></div>

&emsp;&emsp;当前的电脑CUDA的最高版本要求是12.2，所以需要找到不大于12.2版本的Pytorch。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401041455184.png" width=80%></div>

&emsp;&emsp;直接复制对应的命令，进入终端执行即可。这实际上安装的是为 CUDA 12.1 优化的 PyTorch 版本。这个 PyTorch 版本预编译并打包了与 CUDA 12.1 版本相对应的二进制文件和库。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241022153422264.png" width=80%></div>

- **Step 4. 安装Pytorch验证**

&emsp;&emsp;待安装完成后，如果想要检查是否成功安装了GPU版本的PyTorch，可以通过几个简单的步骤在Python环境中进行验证：
```bash
import torch

print(torch.__version__)
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241022154248388.png" width=80%></div>

&emsp;&emsp;如果输出是版本号数字，则表示GPU版本的PyTorch已经安装成功并且可以使用CUDA，如果显示ModelNotFoundError，则表明没有安装GPU版本的PyTorch，或者CUDA环境没有正确配置，此时根据教程，重新检查自己的执行过程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241022154338394.png" width=80%></div>

&emsp;&emsp;当然通过pip show的方式可以很简洁的查看已安装包的详细信息。pip show <package_name> 可以展示出包的版本、依赖关系（展示一个包依赖哪些其他包）、定位包安装位置、验证安装确实包是否正确安装及详情。

- **Step 5. 安装必要的依赖包**

Transfomers是大模型推理时所需要使用的框架，建议使用的版本`Transfomers>=4.37.0`，通过以下指令可以下载最新版本的Transfomers：

pip install transformers -U

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171233368.png" width=80%></div>

安装完成后可以通过以下命令检查是否安装：
```
pip show transformers
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171255682.png" width=80%></div>

接下来需要安装下载工具modelscope以及接下来用来加速模型的训练和部署的库accelerate，通过以下代码进行对应工具的部署：
```
pip install modelscope
pip install accelerate>=0.26.0
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171329493.png" width=80%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029105917193.png" width=80%></div>

- **Step 6. 下载模型文件**

首先我们要创建一个新的文件用于储存下载的代码信息，通过指令`mkdir +name`可以创建一个新的文件夹，创建完毕后用命令`cd +name`可以进入到该文件夹以便于稍后的操作。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029113855227.png" width=50%></div>

通过指令`pwd`可以查看当前所在位置的路径信息（这一步在后续填写调用大模型路径的时候会用到。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029113912171.png" width=50%></div>

在确认进入所在工作文件夹后首先进行模型的项目文件的下载：

```
git clone --recurse-submodules https://github.com/THUDM/GLM-4-Voice
```

下载完成后会存在一个新的文件夹名为GLM-4-Voice，通过以下命令进入该文件夹：

```
cd GLM-4-Voice
```

该文件夹中具有该项目文件的启动文件，全部文件信息如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031154600917.png" width=70%></div>

接下来需要安装依赖，在当前文件夹下执行以下命令进行安装，由于有36个依赖需要安装，会花费较长时间，保持网络的稳定。

```
pip install -r requirements.txt
```

此外，我们还要手动安装一个decoder模型，这是由于Decoder 模型不支持通过 transformers 初始化，所以需要单独下载。

```
git clone https://huggingface.co/THUDM/glm-4-voice-decoder
```

接下来启动模型服务，以下的命令会自动下载一个名为`glm-4-voice-9b`的权重模型文件。

```
python model_server.py --host localhost --model-path THUDM/glm-4-voice-9b
```

或者可以通过以下命令实现int 4量化启动模式，这样做的好处是可以降低推理时所需成本，加快推理速度。

```
python model_server.py --host localhost --model-path THUDM/glm-4-voice-9b --port 10000 --dtype int4 --device cuda:0
```

文件启动后会出现以下信息，说明推理成功了。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241030154628433.png" width=70%></div>

同时，可以后台检测到对应进程开始消耗计算资源，目前有18G的消耗，在全面推理时会有22G左右的消耗（未量化）。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031100218337.png" width=80%></div>

>接下来，请打开一个新的终端窗口，并在其中执行以下步骤

完成模型服务启动以后要开启web服务，通过以下指令便会在下载其余两个所需文件后开启web端的对话服务.

```
python web_demo.py --tokenizer-path  THUDM/glm-4-voice-tokenizer --model-path THUDM/glm-4-voice-9b --flow-path ./glm-4-voice-decoder
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031161232116.png" width=80%></div>

将对应的URL网址信息复制到你任意浏览器上边框的地址栏中，便可实现调用对话。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031161509301.png" width=80%></div>

>这种下载方式需要在github和hugging face上调用资源下载文件，故需要有科学上网的能力。

- **Step 7. 在modelscope源下载模型文件（可选）**

这种办法是一种在执行主机上不需要科学上网的流程，以便于一些特殊环境下实现部署安装（如在服务器环境）。

需要注意的是由于项目文件是在github上代理的，因此模型的启动项目文件需要大家自行手动下载，再通过上传解包到对应文件夹中。通过以下两种方式都可以完成该文件的下载，文件所在地址：

https://github.com/THUDM/GLM-4-Voice

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031162605537.png" width=80%></div>

首先要下载model scope的工具包以便我们在国内环境下完成模型的下载。

```
pip install modelscope
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171329493.png" width=80%></div>

通过以下指令可以进行检查：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028171340209.png" width=80%></div>

接下来在魔搭社区的模型托管页面分别下载对应的文件权重模型、tokenizer模型和decoder模型。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031163948712.png" width=80%></div>

```
modelscope download --model ZhipuAI/glm-4-voice-9b
```

通过以下方式可以指定你所要下载的文件地址，这样即使不在对应文件夹内进行操作也可以实现文件的快速下载。

modelscope download --model ZhipuAI/glm-4-voice-9b  --local_dir ./dir-name

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241028180623990.png" width=80%></div>

同样的方法可以实现tokenizer文件的下载：

```
modelscope download --model ZhipuAI/glm-4-voice-tokenizer
```

以及decoder文件的下载：

```
modelscope download --model ZhipuAI/glm-4-voice-decoder
```

全部文件下载完成后需要启动模型服务以及web文件，通过以下命令实现模型服务的启动，注意这里的`./glm-4-voice-9b`需要修改成你下载好的权重文件所在的地址。

```
python model_server.py --host localhost --model-path ./glm-4-voice-9b
```

或者可以通过以下命令实现int 4量化启动模式，这样做的好处是可以降低推理时所需成本，加快推理速度。

```
python model_server.py --host localhost --model-path ./glm-4-voice-9b --port 10000 --dtype int4 --device cuda:0
```

文件启动后会出现以下信息，说明推理成功了。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241030154628433.png" width=70%></div>

>接下来，请打开一个新的终端窗口，并在其中执行以下步骤

完成模型服务启动以后要开启web服务，这里需要将以下代码中的文件地址分别替换成对应的tokenizer、权重文件、decoder文件所在的地址.

```
python web_demo.py --tokenizer-path  ./glm-4-voice-tokenizer --model-path ./glm-4-voice-9b --flow-path ./glm-4-voice-decoder
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031161232116.png" width=80%></div>

将对应的URL网址信息复制到你任意浏览器上边框的地址栏中，便可实现调用对话。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031161509301.png" width=80%></div>

当然其实有更加一劳永逸的启动办法，通过修改web和server文件中的模型默认调用地址即可永久的实现对本地模型的调用。在修改保存好之后，再次启动该推理时可以只进行`python model_server.py`和`python web.py`即可便捷实现调用。

> web.py

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031145548046.png" width=80%></div>

> model_server.py

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031145726516.png" width=80%></div>

# 4. 线上部署流程

这次为大家介绍的线上部署平台是矩池云，其优势是可以直接在平台连接hugging face等平台，并且对学生或小型研究团体有折扣。

官网连接如下：https://matgo.cn/

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029145028685.png" width=80%></div>

点击网址右上角即可进行登录或注册。每位新用户注册后可领取等值于5元的代金券，具体领取方式请参考下图。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029150324303.png" width=80%></div>

完成注册登录后即可开启在线算力租赁环节，通过图中方式实现进入：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029150626416.png" width=80%></div>

这里推荐的配置是选择`亚太1区`，推理卡选择`4090`，这样的配置有助于我们高效的完成部署推理。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029150744625.png" width=80%></div>

pytorch建议选择2.3.1版本，同时选择高级选项可以设置公钥或自定义端口。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031175150692.png" width=80%></div>

正式启动后会有如下界面，点击`JupyterLab`选项，并点击打开便可启动线上推理。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029160456669.png" width=80%></div>

进入 JupyterLab 后，在主页面会看到如下选项。点击“Terminal”终端以进入命令行界面，开始下载流程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029160558682.png" width=80%></div>

通过以下指令创建一个虚拟环境，名为`glm4voice`，并激活它。

```
conda create -n glm4voice python=3.11
conda activate glm4voice
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029160956418.png" width=80%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029161041960.png" width=80%></div>

可以通过以下指令来升级下载工具 apt，以确保系统使用的是最新版本，提高软件包管理的稳定性和安全性;apt update 会更新本地的软件包索引，确保系统获取到最新的软件版本信息.

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029163800364.png" width=80%></div>

以下代码用于安装和配置 Git LFS（Large File Storage），这是一种扩展工具，帮助 Git 管理和跟踪大型文件,通过 apt 安装 Git LFS 工具，使 Git 能够有效管理大型文件（如图像、视频等），避免大文件对 Git 仓库性能的影响。随后命令初始化 Git LFS，并将其与当前用户的 Git 配置关联。执行此命令后，Git LFS 会自动跟踪并处理被指定为大文件的文件类型，提高 Git 操作的效率和速度。

```
apt install git-lfs
git lfs install
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029163901004.png" width=80%></div>

在矩池云所支持的Linux 系统实例中，网盘对应机器的 /mnt 目录，为永久存储空间，可支持离线数据上传和下载。我们可以在此目录下创建一个新的文件夹，用于保存即将下载的数据。

```
mkdir glm4voice
cd glm4voice
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241031180036621.png" width=80%></div>

运行以下指令以克隆项目:

```
git clone --recurse-submodules https://github.com/THUDM/GLM-4-Voice
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029160817718.png" width=80%></div>

接下来进入项目文件中并实现依赖的安装。

```python
cd GLM-4-Voice 
pip install gradio==4.44.1
```

所有依赖文件如下，可通过需求文档中的信息一步下载所有文件。

```
pip install -r requirements.txt
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029163455516.png" width=80%></div>

接着可以使用 Git 工具拉取 decoder 文件，具体操作如下：

```
git clone https://huggingface.co/THUDM/glm-4-voice-decoder
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029163948969.png" width=80%></div>

下载完成后可以通过指令`ll`来进行查看文件的完整性（网络保持顺畅一般不会出现问题）。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029164017183.png" width=80%></div>

输入以下指令启动` model_server.py`文件，它会先完成权重模型的下载，随后启动模型的推理任务的后端，用于处理推理请求。运行后，服务器将等待来自客户端的请求，对数据进行处理并返回推理结果。

如出现下图的结果，说明推理任务已开启。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029164932242.png" width=100%></div>

接下来，我们需要新建一个终端窗口，然后输入以下指令启动 web.py 文件，以开启 Web 服务，这条命令会运行 web.py 文件，启动一个 Web 服务器，并创建模型的前端接口，使用户可以通过浏览器或 API 访问模型的推理结果。

```
conda activate glm4voice
python web_demo.py --port 8080
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029165046064.png" width=80%></div>

通过命令`nvidia-smi`可以监测显卡的使用情况，可以看到推理部署消耗了21G的显存资源。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029165604575.png" width=80%></div>

此时返回租用页面点击Http端口下的访问连接（URL）即可实现可视页面对话功能。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029155847286.png" width=80%></div>

在页面中，您可以选择进行麦克风输入对话、录音发送或文本发送。完成输入后点击“Submit”按钮即可上传信息并启动推理任务。系统将根据您提供的输入开始处理，并生成相应的推理结果，返回的内容为音频和文字流式输出。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029165104723.png" width=80%></div>

>以下是一些简单的测试：

询问voice-9b模型制作红茶的流程，可以得到流畅且准确的结果，这表明该模型具有足够的世界知识和人类对齐能力。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029173110745.png" width=80%></div>

让大模型用天津方言生成一段绕口令，指令遵循能力和音色效果都很不错。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029165811742.png" width=80%></div>

用陕西方言进行测试，虽然模型指令遵循能力不错，但是并没有陕西话这个语音能力，想复现陕北说书就略有遗憾了。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029170422135.png" width=80%></div>

该模型的音频理解能力也十分令人欣喜，它会在接受信息后进行推理对话，就如同数字人一般实现语言交互。以下便是让它去理解刚刚生成的天津话绕口令所返回的信息。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029172255036.png" width=80%></div>

该模型同样还支持英文，通过测试将句子--知识就是力量，法国培根。进行翻译指令的输出，可以得到一段非常优美的英文朗读，并对该格言进行阐述解读，这表明了glm4-voice-9b具有双语能力以及足够的推理逻辑能力。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029172718159.png" width=80%></div>

可以观察到，模型的输入通常分为三个主要部分：系统（system）、用户（user）和助手（assistant）。这三个部分构成了对大模型的整体提示（prompt），为模型的响应提供了明确的上下文和框架。

首先，系统部分设定了模型的基本行为和特性，通常包含了一些规则和指导方针，以确保模型在交互过程中保持一致性和相关性。其次，用户部分是用户输入的内容，它反映了用户的意图和需求，直接影响到模型生成的回复。最后，助手部分则是模型基于系统和用户输入所产生的响应，它体现了模型对前两者的理解与处理。

这种结构化的输入格式不仅提升了模型的生成质量，还使得模型能够更好地适应不同的应用场景和用户需求。通过清晰的分块，开发者能够更方便地调整和优化模型的表现，从而提升用户体验。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029172805876.png" width=80%></div>

模型的输出是遵守着13文本tokens和16音频tokens交替输出的，这样做的好处是在推理过程中就可以不间断的产出音频，同时最后还能看到文字信息。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029172815399.png" width=80%></div>

**模型保存**

在结束模型推理后记得保存并结束租赁任务，否则继续使用线上时长会导致持续计费哦，在模型信息页面点击`更多`，再点击保存到个人环境。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029173504669.png" width=80%></div>

此时会显示以下弹窗，可以选择设置项目的名称，以及保存后自动释放机器选项，在保存成功后下次即可快速从保存环境启动。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241029173818523.png" width=80%></div>

# 5. 开源大模型比拼

在当前大模型快速发展的背景下，各类模型的水平参差不齐，媒体宣传往往夸大其词，而一些公司则过分标榜自身模型的能力，声称“达到ChatGPT水平”或“国内第一”。这导致外行人难以分辨真实情况。业界迫切需要建立一个公开、公正和公平的大模型评测系统，以真实展现各类模型的优缺点，从而帮助行业把握发展水平和与国际技术的差距，为未来的努力方向提供清晰的指导。

对于不具备大模型研发能力的公司而言，了解技术边界并高效进行技术选型显得尤为重要。一个有效的评测系统可以避免重复开发和不必要的技术争论，促进更为顺畅的沟通。此外，对于研发人员和学术界人士来说，各类模型的效果对比能够反映出不同技术路线的有效性，从而提供参考，减少资源浪费，推动整个大模型生态的高效发展。

虽然开源模型本质上下载和使用都是免费的，但是这还有一些隐形成本，比如由模型参数大小带来的硬件推理成本，以及不同基座模型能力不同导致后期微调/应用时所需的成本也不同，选择一个合适任务环境的模型有时候比选择一个分数榜高的模型有时更为重要，本章节摘录了第三方的测评结果，力求为大家带来一个相对公平的测试结果。

CLiB中文大模型能力评测榜单：https://github.com/jeinlee1991/chinese-llm-benchmark?tab=readme-ov-file

>MMLU - 一种针对大模型的语言理解能力的测评，是目前最著名的大模型语义理解测评之一，任务涵盖的知识很广泛，语言是英文，用以评测大模型基本的知识覆盖范围和理解能力。

>C Eval - C-Eval 是一个全面的中文基础模型评估套件。它包含了13948个多项选择题，涵盖了52个不同的学科和四个难度级别。用以评测大模型中文理解能力。

>AGI Eval - 微软发布的大模型基础能力评测基准，主要评测大模型在人类认知和解决问题的一般能力，涵盖全球20种面向普通人类考生的官方、公共和高标准录取和资格考试，包含中英文数据。

>GSM8K - OpenAI发布的大模型数学推理能力评测基准，涵盖了8500个中学水平的高质量数学题数据集。数据集比之前的数学文字题数据集规模更大，语言更具多样性，题目也更具挑战性。

>10B以下开源模型榜单

优点
- 计算资源需求低：相较于大型模型，10B以下的模型在计算资源和存储需求上更为轻量，适合在资源有限的设备上运行，例如移动设备和边缘计算设备。

- 更快的推理速度：较小的模型在推理时的响应速度较快，适合实时应用场景，如聊天机器人、语音助手和即时翻译等。

- 训练成本较低：训练10B以下的模型所需的时间和计算资源相对较少，使得开发和实验的成本更低，更适合中小型企业和科研机构。

- 适应性强：小模型可以更容易地进行微调和迁移学习，使得它们能够快速适应新的任务或领域。

- 更易于部署：由于其较小的体积和低资源需求，10B以下的模型可以更方便地部署到云端、移动端和嵌入式设备上。

适合的应用场景
- 聊天机器人和虚拟助手：适合用于自然语言处理的基本任务，如文本生成、问答和情感分析。

- 文本分类和情感分析：在情感分析、主题分类和评论过滤等任务中表现良好。

- 语言翻译：可以用于简单的翻译任务，尤其是针对特定领域（如旅游、餐饮等）的翻译。

- 信息提取和摘要：适合从较短文本中提取关键信息或生成摘要。

- 教育和辅助学习：用于个性化学习助手或教育工具，帮助学生进行知识巩固。

- 实验和快速原型：在研究和开发初期阶段，可以使用小模型进行实验和原型设计，快速验证想法。

- 文本生成和创作：适用于创作短篇文章、社交媒体内容或自动回复等场景。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101140526238.png" width=80%></div>

>10~20B 开源模型榜单

优点
- 良好的性能平衡：相较于较小的模型，这一参数范围的模型通常在语言理解和生成能力上有显著提升，能够处理更复杂的任务。

- 更丰富的上下文理解：这些模型可以处理更长的上下文信息，适合需要上下文理解和连贯性的任务，如长篇文章的生成或对话系统。

- 多任务学习能力：在一个模型中训练多种任务，能够实现更好的泛化能力，适合需要同时完成多项任务的应用。

- 高效的微调能力：由于模型参数相对较多，能够更好地适应不同领域和任务的微调，尤其是在数据量适中的情况下。

- 对抗性训练：可以更有效地进行对抗性训练，提高模型的鲁棒性，使其在面对噪声或对抗样本时表现更佳。

适合的应用场景
- 对话系统：适合用于更复杂的聊天机器人和虚拟助手，能够提供更自然、更智能的对话体验。

- 文本生成：用于长篇文章、故事创作和其他需要深入内容生成的场景，能够保持一致的风格和主题。

- 翻译和转述：在多语言处理和内容转述中表现出色，能够捕捉细微的语言差异。

- 信息检索和推荐系统：适合用于构建智能搜索引擎和推荐系统，能够理解用户查询的复杂性。

- 数据增强和合成：用于生成训练数据，帮助模型更好地适应小样本学习或少样本学习场景。

- 情感分析和社会网络分析：适用于分析社交媒体内容和用户情感，能够从复杂文本中提取深层次的情感信息。

- 医疗和法律领域的文本处理：在特定领域中提供高质量的文本处理和信息提取，帮助专业人士更高效地处理大量信息。

- 研究和开发：作为实验模型，用于探索新算法、新架构或新应用的研究。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101141614988.png" width=80%></div>

>30B 以上大模型排行榜

优点
- 卓越的语言理解和生成能力：这些大型模型通常在自然语言处理任务中展现出卓越的性能，能够生成更自然、更连贯的文本。

- 丰富的知识基础：由于训练数据量大，模型通常具备广泛的知识背景，能够处理多领域的问题并生成相关的内容。

- 增强的上下文处理能力：较大的参数量使得模型能够处理更长的上下文，适合复杂的对话和文本生成任务。

- 更强的推理和逻辑能力：在需要深度推理和逻辑分析的任务中，大模型展现出更强的能力，能够执行复杂的推理链。

- 高效的迁移学习能力：能够在不同的任务和领域之间迁移知识，适合快速适应新的应用场景。

- 鲁棒性和适应性：在面对不确定性和噪声时，表现出更高的鲁棒性，适合处理复杂和动态环境中的任务。

适合的应用场景
- 复杂对话系统：可用于构建高度智能的对话代理，能够进行长时间和多轮的自然对话。

- 高质量文本生成：适合长篇内容的生成，如小说、剧本、研究论文等，确保语义连贯和风格一致。

- 多模态应用：在结合图像、音频等多种数据源时表现出色，如视觉问答和图像描述生成。

- 知识问答系统：能够从广泛的知识库中提取信息，支持复杂的问答任务。

- 个性化推荐系统：在分析用户偏好和行为数据时，生成更精准的个性化内容和推荐。

- 高级文本分析：适用于法律、医疗等行业中的复杂文本分析与信息提取，帮助专业人士处理复杂文档。

- 科学研究和创新：为学术研究提供强大的支持，帮助科学家进行数据分析、文献综述和理论建模。

- 跨语言处理：支持多语言的理解和生成，适合国际化应用和全球市场的需求。

- 大规模自动化任务：在企业环境中，自动化处理复杂的业务流程，如客户支持、内容审核等。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101141652874.png" width=60%></div>

**信息抽取排行榜：**

示例
文本：“招商银行5亿元，建设银行增长约3.5亿元，农业银行约2亿元。”

提取出的组织机构名称：

招商银行
建设银行
农业银行

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142002936.png" width=60%></div>

**分类能力排行榜:**

将以下10个单词分类为动物或植物。
树木、狮子、玫瑰、草地、松鼠、猴子、蘑菇、兔子、山羊、香蕉

动物：狮子、松鼠、猴子、兔子、山羊。

植物：树木、玫瑰、草地、蘑菇、香蕉。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142023756.png" width=60%></div>

**阅读理解能力排行榜：**


阅读理解能力是一种符合能力，考查针对给定信息的理解能力。 依据给定信息的种类，可以细分为：文章问答、表格问答、对话问答……

评测样本举例：

牙医：好的，让我们看看你的牙齿。从你的描述和我们的检查结果来看，你可能有一些牙齦疾病，导致牙齿的神经受到刺激，引起了敏感。此外，这些黑色斑点可能是蛀牙。
病人：哦，真的吗？那我该怎么办？
牙医：别担心，我们可以为你制定一个治疗计划。我们需要首先治疗牙龈疾病，然后清除蛀牙并填充牙洞。在此过程中，我们将确保您感到舒适，并使用先进的技术和材料来实现最佳效果。
病人：好的，谢谢您，医生。那么我什么时候可以开始治疗？
牙医：让我们为您安排一个约会。您的治疗将在两天后开始。在此期间，请继续刷牙，使用牙线，并避免吃过于甜腻和酸性的食物和饮料。
病人：好的，我会的。再次感谢您，医生。
牙医：不用谢，我们会尽最大的努力帮助您恢复健康的牙齿。
基于以上对话回答：病人在检查中发现的牙齿问题有哪些？

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142138038.png" width=60%></div>

**数据分析排行榜：**


专门考查大模型对表格的理解分析能力，常用于数据分析。
评测样本举例：

```
李四, 27, 男, 中国, 178, 72, 本科
Anna, 34, 女, 瑞典, 170, 60, 硕士
Carlos, 39, 男, 阿根廷, 177, 78, 博士
Yuki, 22, 女, 日本, 158, 48, 高中
John, 31, 男, 加拿大, 180, 74, 硕士
Sofia, 30, 女, 俄罗斯, 165, 62, 本科
David, 45, 男, 英国, 183, 85, 博士
问题：身高最矮的是哪国人？
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142155873.png" width=60%></div>

**中文指令遵循能力：**

顾名思义，旨在监测大模型的中文指令理解和执行能力，例子：

写一篇至少150字的博客，介绍睡在吊床上的好处。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142402308.png" width=60%></div>

**数学能力：**

```
考查大模型的数学基础能力可以设计以下测试题目，涵盖1000以内的整数加减法和不超过2位有效数字的浮点数加减乘除。

整数加减法题目：
378 + 124 = ？
850 - 432 = ？
512 + 249 = ？
640 - 215 = ？
789 + 156 + 34 = ？
浮点数加减乘除题目：
0.56 + 0.75 = ？
0.45 - 0.2 = ？
1.2 * 0.3 = ？
0.99 / 0.3 = ？
0.85 + 0.4 / 0.25 = ？
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142114988.png" width=60%></div>

**编程能力**

本着以下两个测试指标进行了全模型的排行：

HumanEval是OpenAI于2021年推出的一个用于评估代码生成模型性能的数据集，包含164个手工编写的编程问题。每个问题提供了函数签名、文档字符串及多个单元测试，涵盖语言理解、推理、算法和简单数学等方面，难度与简单的软件面试问题相当。该数据集的特点在于，生成的代码不仅需语法正确，还需通过所有相关的单元测试，确保功能正确性。评估结果通过Pass@k指标表示，k表示一次性生成不同答案的数量中至少有1个正确答案的比例，例如Pass@1和Pass@10。

MBPP（Mostly Basic Programming Problems）是谷歌于2021年推出的一个数据集，包含974个设计用于初级程序员的短小Python函数问题。该数据集提供每个问题的文本描述和功能正确性测试用例，评估方式与HumanEval类似，通过Pass@k指标衡量模型生成的答案准确性。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101151957378.png" width=60%></div>

该榜单来自dataleaner：https://www.datalearner.com/ai-models/leaderboard/datalearner-llm-leaderboard

**综合能力排行**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101142216853.png" width=60%></div>

## 其他模型榜单推荐


**司南Opencompass**

>该平台同时支持模型竞技场进行模型能力对比测试。

官网地址：https://rank.opencompass.org.cn/home

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101152755447.png" width=60%></div>

**数据学习**

该网站同时提供有关AI的新闻咨询，适合学术研究学习。

官网地址：https://www.datalearner.com/ai-models/leaderboard/datalearner-llm-leaderboard

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241101153525709.png" width=60%></div>